In [ ]:
%config IPCompleter.greedy=True
from kiteconnect import KiteConnect
import json, time

_api_key = 'your api key'
_api_secret = 'your api secret'
enter_request_token_message = 'Enter request token: '
access_token_key = 'access_token'
kite = KiteConnect(api_key=_api_key)
print(kite.login_url())
request_token = input(enter_request_token_message)
request_access_token = kite.generate_session(request_token, _api_secret)
kite.set_access_token(request_access_token[access_token_key])

In [1]:
interested_shares = ['AXISBANK','ITC','LT']

REJECTED = 'REJECTED'
COMPLETE = 'COMPLETE'
CANCELLED = 'CANCELLED'
OPEN = 'OPEN'
BUY = 'BUY'
SELL = 'SELL'
MARKET = 'MARKET'
LIMIT = 'LIMIT'
regular = 'regular'
NSE = 'NSE'
CNC = 'CNC'
fileName = './myPrecious/stock_info.json'
holding_tag = 'holding'
startingMoneyPerShare = 2000
exceptions_limit = 200
status_key = 'status'
name_key = 'name'
available_balance_key = 'availableBalance'
owning_shares_key = 'owningShares'
interested_price_key = 'interestedPrice'
interested_buying_price_key = 'interestedBuyingPrice'
interested_selling_price_key = 'interestedSellingPrice'
cost_price_key = 'costPrice'
selling_price_key = 'sellingPrice'
stocks_key = 'stocks'
trading_symbol_key = 'tradingsymbol'
existing_order_quantity_key = 'existingOrderQuantity'
ltp_key = 'ltp'
last_price_key = 'last_price'
realised_quantity_key = 'realised_quantity'
quantity_key = 'quantity'
t1_quantity_key = 't1_quantity'
average_price_key = 'average_price'
transaction_type_key = 'transaction_type'
tag_key = 'tag'
simple_moving_average_key = 'simpleMovingAverage'
sum_key = 'sum'
price_history_key = 'priceHistory'
keys_to_ignore_while_printing = [simple_moving_average_key, sum_key, price_history_key, tag_key]
open_key = 'open'
close_key = 'close'
low_key = 'low'
high_key = 'high'
candlesticks_key = 'candlesticks'
points_per_candlestick = 30
candlesticks_limit = 20
datapoints_limit = points_per_candlestick * candlesticks_limit
selling_price_factor = 1.01
buying_price_factor = 0.995
available_money_factor = 1.008

In [45]:
# Never run more than once

counter = 0
exceptions_count = 0
ta = {} # ta means technical analysis
for share in interested_shares:
    ta[share] = {}
    ta[share][price_history_key] = []
    ta[share][sum_key] = 0
    ta[share][simple_moving_average_key] = 0

In [46]:
#Utility functions

def roundPrice(price, base=0.05):
    return round(base*round(price/base),2)

In [47]:
# Printing functions

def print_all_shares_info(shares):
    print('{:14}{:14}{:14}{:14}{:14}{:14}{:14}{:14}{:14}'
          .format('SYMBOL','OWNING','AVG. PRICE','EX. ORDER','LTP','INT. PP', 'INT. SP', 'TRADED', 'MONEY'))
    print('='*118)
    for sym in shares:
        one_share = ''
        for key in shares[sym]:
            if key not in keys_to_ignore_while_printing:
                one_share = one_share + '{:<14}'.format(shares[sym][key])
        print(one_share)
        

# BUY     20      of      POWERGRID      at       Rs. 148.5    each
def print_order_info(_ttype, _quan, _sym, _price):
    print('{:<8}{:<8}{:<8}{:<15}{:<8} Rs. {:<8} each'
          .format(_ttype, _quan, 'of', _sym, 'at', _price))
    
        
        
def printOpenOrders():
    all_orders = kite.orders()
    open_orders = []
    for order in all_orders:
        if order[status_key] == open_order:
            open_orders.append(order)
    print(open_orders)

In [48]:
def createBuyOrSellOrders():
    if (exceptions_count <= exceptions_limit):
        count_of_orders_placed = 0
        for key in all_shares_info:
            if(canSell(all_shares_info[key]) and shouldSell(all_shares_info[key])):
                placeSellOrder(all_shares_info[key])
                count_of_orders_placed = count_of_orders_placed + 1
            elif(canBuy(all_shares_info[key]) and shouldBuy(all_shares_info[key])):
                placeBuyOrder(all_shares_info[key])
                count_of_orders_placed = count_of_orders_placed + 1

        # If an order has been placed, then something was bought or sold.
        # So, update the numberOfTimesTraded for that share; and sleep for sometime
        if(count_of_orders_placed > 0):
            time.sleep(20)
            populateTotalNumberOfTimesTraded(all_shares_info)
    else:
        print("TOO MANY EXCEPTIONS HAVE OCCURRED. SKIPPED PLACING AN ORDER.")
        print("TO RESUME, FIX THE CAUSE AND RESET THE EXCEPTION COUNT")

In [56]:
def canBuy(stock):
    # Can buy a stock only if you have money to buy a stock
    if(stock[available_balance_key] >= stock[interested_buying_price_key]):
        return True
    return False


def shouldBuy(stock):
    return False
    # Should buy a stock only if there is no existing order for it
    should_buy = stock[existing_order_quantity_key] == 0
    # Should buy a stock only if you own zero units of it. 
    # Negative values causes double buy orders to be placed. So, strict zero check. 
    should_buy = should_buy and stock[owning_shares_key] == 0
    #should_buy = should_buy and len(ta[stock[trading_symbol_key]][price_history_key]) >= datapoints_limit
    return should_buy


def canSell(stock):
    # Can sell a stock only if you own shares of it
    if(stock[owning_shares_key] > 0):
        return True
    return False


def shouldSell(stock):
    # Should sell a stock only if there is no existing order for it
    should_sell = True and (stock[existing_order_quantity_key] == 0)
    # In case average price computation goes wrong, the below check should be able to catch it
    should_sell = should_sell and (stock[average_price_key] > 0.9*stock[ltp_key]) 
    return should_sell

In [57]:
def placeBuyOrder(stock):
    quan = (int)(stock[available_balance_key]/stock[interested_buying_price_key])
    try:  
        kite.place_order(
            variety=regular,
            exchange=NSE,
            tradingsymbol=stock[trading_symbol_key],
            transaction_type=BUY,
            quantity=quan,
            product=CNC,
            order_type=LIMIT,
            price=stock[interested_buying_price_key]
            )
        print_order_info(BUY, quan, stock[trading_symbol_key], stock[interested_buying_price_key])
    except Exception as e:
        exceptions_count = exceptions_count + 1
        print(stock[trading_symbol_key] + " : BUY --> Exception occurred: " + str(e))
        time.sleep(1)
    
    
def placeSellOrder(stock):
    quan = stock[owning_shares_key]
    _tag = stock[tag_key]
    try:  
        kite.place_order(
            variety=regular,
            exchange=NSE,
            tradingsymbol=stock[trading_symbol_key],
            transaction_type=SELL,
            quantity=quan,
            product=CNC,
            order_type=LIMIT,
            price=stock[interested_selling_price_key],
            tag=_tag
            )
        print_order_info(SELL, quan, stock[trading_symbol_key], stock[interested_selling_price_key])
    except Exception as e:
        exceptions_count = exceptions_count + 1
        print(stock[trading_symbol_key] + " : SELL --> Exception occurred: " + str(e))
        time.sleep(1)

In [58]:
# The order of execution of these is important. Do take a closer look while changing.
def initializeSharesInfo():
    stocks_info = {}
    populateTradingSymbol(stocks_info)
    initializeKeysWithDummyValues(stocks_info)
    populateHoldingAndAveragePrice(stocks_info)
    populateCurrentOrderInfo(stocks_info)
    populateLtp(stocks_info)
    doTechnicalAnalysis(stocks_info)
    populateInterestedBuyingPrice(stocks_info)
    populateInterestedSellingPrice(stocks_info)
    populateNumberOfTimesSoldTillYesterday(stocks_info)
    populateTotalNumberOfTimesTraded(stocks_info)
    populateAvailableBalanceInfo(stocks_info)
    return stocks_info
          

# Number of shares that can be sold            
def populateHoldingAndAveragePrice(stocks_info):
    current_holdings = kite.holdings()
    for holding in current_holdings:
        symbol = holding[trading_symbol_key]
        if(symbol in stocks_info.keys()):
            stocks_info[symbol][owning_shares_key] = holding[quantity_key] + holding[t1_quantity_key]
            stocks_info[symbol][average_price_key] = holding[average_price_key]
            stocks_info[symbol][tag_key] = holding_tag
    # In case there is a holding and a current order has already been placed
    _updateHoldingAndAveragePrice(stocks_info)
    
    
def _updateHoldingAndAveragePrice(stocks_info):
    current_orders = kite.orders()
    for order in current_orders:
        if order[status_key] == COMPLETE and order[tag_key] != holding_tag:
            if order[transaction_type_key] == BUY and order[trading_symbol_key] in stocks_info.keys():
                prev_quantity = stocks_info[order[trading_symbol_key]][owning_shares_key]
                prev_avg_price = stocks_info[order[trading_symbol_key]][average_price_key]
                stocks_info[order[trading_symbol_key]][owning_shares_key] = prev_quantity + order[quantity_key]
                if stocks_info[order[trading_symbol_key]][owning_shares_key] > 0:
                    stocks_info[order[trading_symbol_key]][average_price_key] \
                    = roundPrice((prev_quantity*prev_avg_price 
                       + order[quantity_key]*order['price'])/stocks_info[order[trading_symbol_key]][owning_shares_key])
            elif order[transaction_type_key] == SELL and order[trading_symbol_key] in stocks_info.keys():
                prev_quantity = stocks_info[order[trading_symbol_key]][owning_shares_key]
                prev_avg_price = stocks_info[order[trading_symbol_key]][average_price_key]
                stocks_info[order[trading_symbol_key]][owning_shares_key] = prev_quantity - order[quantity_key]


# Existing orders which are not executed    
def populateCurrentOrderInfo(stocks_info):
    current_orders = kite.orders()
    for order in current_orders:
        if(order[status_key] == OPEN and order[trading_symbol_key] in stocks_info.keys()):
            stocks_info[order[trading_symbol_key]][existing_order_quantity_key] = order[quantity_key]

            
# When new orders are executed, update the number of times the stocks have been traded
def populateTotalNumberOfTimesTraded(stocks_info):
    for key in stocks_info:
        stocks_info[key][times_sold_key] = stocks_info[key][times_sold_till_yesterday_key]
    current_orders = kite.orders()
    for order in current_orders:
        if order[status_key] == COMPLETE:
            if order[transaction_type_key] == SELL and order[trading_symbol_key] in stocks_info.keys():
                sym = order[trading_symbol_key]
                stocks_info[sym][times_sold_key] = stocks_info[sym][times_sold_key] + 1 
            
            
def initializeKeysWithDummyValues(stocks_info):
    for key in stocks_info:
        stocks_info[key][owning_shares_key] = 0
        stocks_info[key][average_price_key] = 0
        stocks_info[key][existing_order_quantity_key] = 0
        stocks_info[key][ltp_key] = 0
        stocks_info[key][interested_buying_price_key] = 0
        stocks_info[key][interested_selling_price_key] = 0
        stocks_info[key][times_sold_till_yesterday_key] = 0
        stocks_info[key][times_sold_key] = 0
        stocks_info[key][available_balance_key] = 0
        stocks_info[key][tag_key] = ''

In [59]:
# DO NOT CHANGE THE LOGIC IN THESE FUNCTIONS. THEY WORK WELL.

def populateInterestedBuyingPrice(stocks_info):
    for key in stocks_info:
        stocks_info[key][interested_buying_price_key] \
        = roundPrice(buying_price_factor * min(ta[key][simple_moving_average_key], stocks_info[key][ltp_key]))
        

def populateInterestedSellingPrice(stocks_info):
    for key in stocks_info:
        stocks_info[key][interested_selling_price_key] \
        = roundPrice(max(selling_price_factor*stocks_info[key][average_price_key],stocks_info[key][ltp_key]))
        
        
def populateLtp(stocks_info):
    instruments_to_fetch_ltp = []
    for key in stocks_info:
        instruments_to_fetch_ltp.append('NSE:' + key)
    ltps = kite.ltp(instruments_to_fetch_ltp)
    for key in stocks_info:
        stocks_info[key][ltp_key] = ltps['NSE:' + key][last_price_key]
        

def populateTradingSymbol(stocks_info):
    for val in interested_shares:
        stocks_info[val] = {trading_symbol_key : val} 
        

def populateAvailableBalanceInfo(stocks_info):
    for key in stocks_info:
        if(stocks_info[key][owning_shares_key] == 0):
            stocks_info[key][available_balance_key] \
            = roundPrice(startingMoneyPerShare * (available_money_factor**stocks_info[key][times_sold_key]))
        else:
            stocks_info[key][available_balance_key] = 0
            

def populateNumberOfTimesSoldTillYesterday(stocks_info):
    with open(fileName) as json_file:
        all_stocks = json.load(json_file)
        for stock_key in stocks_info:
            stocks_info[stock_key][times_sold_till_yesterday_key] = all_stocks[stock_key][times_sold_key]

In [60]:
def doTechnicalAnalysis(stocks_info):
    if counter < datapoints_limit:
        _do_ta_len_less_than_datapoints_limit(stocks_info)
        # Here len refers to the length of price history array
    else:
        _do_ta_len_is_equal_to_datapoints_limit(stocks_info)
    

def _do_ta_len_less_than_datapoints_limit(stocks_info):
    for key in stocks_info:
        ta[key][price_history_key].append(stocks_info[key][ltp_key])
        ta[key][sum_key] = ta[key][sum_key] + stocks_info[key][ltp_key]
        ta[key][simple_moving_average_key] \
        = roundPrice(ta[key][sum_key]/len(ta[key][price_history_key]))
    

def _do_ta_len_is_equal_to_datapoints_limit(stocks_info):
    for key in stocks_info:
        ta[key][sum_key] = ta[key][sum_key] \
        - ta[key][price_history_key][counter%datapoints_limit] \
        + stocks_info[key][ltp_key]
        ta[key][price_history_key][counter%datapoints_limit] = stocks_info[key][ltp_key]
        ta[key][simple_moving_average_key] = \
        roundPrice(ta[key][sum_key]/len(ta[key][price_history_key]))
    
    #if candlesticks_key in ta['INFY'].keys():
     #   update_candlesticks()
    #else:
     #   initialize_candlesticks()
            
            
def initialize_candlesticks():
    for key in ta:
        candle_stick_num = 0
        while(candle_stick_num < candlesticks_limit):
            working_list = ta[key][price_history_key][candle_stick_num*points_per_candlestick:points_per_candlestick*(candle_stick_num + 1)]
            ta[key][candlesticks_key].append([working_list[0],max(working_list),min(working_list),working_list[-1]]) 
            candle_stick_num = candle_stick_num + 1
            

def update_candlesticks():
    if counter%datapoints_limit == 0:
        candle_stick_num = (counter/datapoints_limit)%no_of_candlesticks
        for key in ta:
            working_list = ta[key][price_history_key][candle_stick_num*points_per_candlestick:points_per_candlestick*(candle_stick_num + 1)]
            ta[key][candlesticks_key][candle_stick_num] = [working_list[0],max(working_list),min(working_list),working_list[-1]]

In [ ]:
all_shares_info = initializeSharesInfo()
print_all_shares_info(all_shares_info)

In [ ]:
temp = 1
all_shares_info = initializeSharesInfo()
while(temp >= 1):
    all_shares_info = initializeSharesInfo()
    createBuyOrSellOrders()
    temp = temp + 1
    counter = counter + 1